In [2]:
import csv
import math
import random
import pandas as pd

In [63]:
import csv

def loadCsv(filename):
    with open("C:/Users/karan/Downloads/diabetes.csv", 'r') as file:
        lines = csv.reader(file)
        dataset = list(lines)
        
        # Skip header row
        header = dataset[0]
        dataset = dataset[1:]
        
        for i in range(len(dataset)):
            dataset[i] = [float(x) for x in dataset[i]]
        
        return dataset


In [88]:
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset)  * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [89]:
def separatedByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [90]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [91]:
def stdev(numbers):
    avg = mean(numbers)

    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)

    return math.sqrt(variance)



In [92]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]

    del summaries[-1]

    return summaries



In [100]:
def summarizeByClass(dataset):
    separated = separatedByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [101]:
def calculateProbability(x, mean, stdev):

    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))

    return (1/(math.sqrt(2*math.pi)*stdev))*exponent

In [102]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    
    return probabilities


In [103]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    
    bestLabel, bestProb = None, -1
    
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    
    return bestLabel


In [104]:
def getPredictions(summaries, testSet):
    predictions = []
    
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    
    return predictions


In [105]:
def getAccuracy(testSet, predictions):
    correct = 0
    
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    
    return (correct / float(len(testSet))) * 100.0


In [106]:
def main():
    filename = 'pima-indians-diabetes.data.csv'
    splitRatio = 0.67
    
    dataset = loadCsv(filename)
    
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    
    print('Split {0} rows into train = {1} and test = {2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
    
    # Prepare model
    summaries = summarizeByClass(trainingSet)
    
    # Test model
    predictions = getPredictions(summaries, testSet)
    
    accuracy = getAccuracy(testSet, predictions)
    
    print('Accuracy: {0}%'.format(accuracy))

main()


Split 768 rows into train = 514 and test = 254 rows
Accuracy: 75.59055118110236%
